In [1]:
import torch
import data_loader
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
import torch.optim as optim

In [69]:
trainx, devx, testx, trainy, devy, testy = data_loader.load_all_classic_random_split(flatten=False)

Processing albert
Processing canon_12_5
Processing chen_12_7
Processing daniel
Processing haobin_11_22
Processing isa_12_5
Processing janet
Processing joanne
Processing jq_12_6
Processing kelly_11_7
Processing kevin_11_7
Processing ruocheng
Processing russell_11_20_stand
Processing russell_11_7
Processing russell_random_12_7
Processing solomon
Processing wenzhou_12_5
Processing yiheng_11_30
Processing yiheng_12_5
Processing yongxu_11_30
Processing zhaoye_12_7
Splitting out test set
Splitting out dev and train set


In [70]:
trainx, trainy = data_loader.augment_train_set(trainx, trainy, augment_prop=3, is_flattened=False)
trainx.shape, devx.shape, testx.shape, trainy.shape, devy.shape, testy.shape

Augmenting TRAIN set with proportion 3


((33280, 100, 3), (1040, 100, 3), (1041, 100, 3), (33280,), (1040,), (1041,))

In [63]:
BATCH_SIZE = 500

def get_dataloader(x, y, batch_size):
    dataset = [(x[i].T, y[i]) for i in range(y.shape[0])]
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)
    return dataloader

trainloader = get_dataloader(trainx, trainy, BATCH_SIZE)
devloader = get_dataloader(devx, devy, 1)
testloader = get_dataloader(testx, testy, 1)

In [64]:
_, num_feature, num_channel = trainx.shape
num_feature, num_channel

(100, 3)

In [65]:
def acc(data_loader):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in data_loader:
            x, y = data
            outputs = net(x.float())
            _, predicted = torch.max(outputs.data, 1)
            
            w = torch.sum((predicted - y) != 0).item()
            r = len(y) - w
            correct += r 
            total += len(y)
    return correct / total

In [66]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv1d(
            in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1
        )  
        # 16 channel, num_feature
        self.pool1 = nn.MaxPool1d(kernel_size=3, stride=1, padding=1)
        # 16 channel, num_feature

        self.conv2 = nn.Conv1d(
            in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=2
        )
        # 32 channel, num_feature
        self.pool2 = nn.MaxPool1d(kernel_size=5, stride=1, padding=2)
        # 32 channel, num_feature

        self.fc1 = nn.Linear(num_feature * 32, 2000)
        self.dropout = nn.Dropout(p=0.1)
        self.fc2 = nn.Linear(2000, 500)
        self.fc3 = nn.Linear(500, 26)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = x.view(-1, num_feature * 32)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
net = Net()

In [67]:
criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(net.parameters(), lr=0.00001, momentum=0.9)
optimizer = optim.Adam(net.parameters(), lr=0.0001)

for epoch in range(10):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader):
        print(f'{i if i%20==0 else ""}.', end='')

        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs.float())
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    trainacc = acc(trainloader)
    devacc = acc(devloader)

    print('')
    print(f'Epoch {epoch} trainacc={trainacc} devacc={devacc}')

print('Finished Training')

0....................................................................
Epoch 0 trainacc=0.36867727380669674 devacc=0.3783783783783784
0....................................................................
Epoch 1 trainacc=0.5356803609593921 devacc=0.4508108108108108
0....................................................................
Epoch 2 trainacc=0.6546544763714082 devacc=0.4864864864864865
0....................................................................
Epoch 3 trainacc=0.7317442412728568 devacc=0.4627027027027027
0............

KeyboardInterrupt: 

In [68]:
acc(testloader)

0.15749525616698293